In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# 1. GEE 튜토리얼

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [ ]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='energylife-458603')

## Test the API

Test the API by printing the elevation of Mount Everest.

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


## Map visualization

`ee.Image` objects can be displayed to notebook output cells. The following two
examples demonstrate displaying a static image and an interactive map.


### Static image

The `IPython.display` module contains the `Image` function, which can display
the results of a URL representing an image generated from a call to the Earth
Engine `getThumbUrl` function. The following cell will display a thumbnail
of the global elevation model.

In [ ]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

### Interactive map

The [geemap](https://github.com/gee-community/geemap)
library can be used to display `ee.Image` objects on an interactive
[ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) map.

The following cell provides an example of using the `geemap.Map` object to
display an elevation model.

In [ ]:
# Import the geemap library.
import geemap

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a map object.
m = geemap.Map(center=[20, 0], zoom=3)

# Add the elevation model to the map object.
m.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Display the map.
display(m)

## Chart visualization

Some Earth Engine functions produce tabular data that can be plotted by
data visualization packages such as `matplotlib`. The following example
demonstrates the display of tabular data from Earth Engine as a scatter
plot. See [Charting in Colaboratory](https://colab.sandbox.google.com/notebooks/charts.ipynb)
for more information.

In [ ]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat TOA image.
img = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_034033_20000913')

# Select Red and NIR bands and sample 500 points.
samp_fc = img.select(['B3','B4']).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()

# 2. 우주베키스탄 매립지 영역 인공위성 영상

# energyLife-458603
### 인공위성 영상 바로 띄우기

In [8]:
# 코랩에서 바로 띄우기
# 필요한 라이브러리 설치
# !pip install earthengine-api geemap --quiet

# Earth Engine 인증 및 초기화
import ee
import geemap

# 인증 → 처음 한 번만 필요
ee.Authenticate()

# 프로젝트 ID: GCP 콘솔에서 확인한 것 사용 (예: energylife-458603)
ee.Initialize(project='energylife-458603')  # ← 여기에 정확한 ID 입력

# 관심 지점과 분석 영역(AOI) 정의
point = ee.Geometry.Point([67.959741, 40.098217])
aoi = point.buffer(500).bounds()  # 500m 반지름 -> 가로 세로 1km

# Sentinel-2에서 연도별 여름 영상 추출
def get_s2_rgb(year):
    start = ee.Date(f"{year}-03-01")
    end = ee.Date(f"{year}-05-30")
    image = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
             .filterBounds(aoi)
             .filterDate(start, end)
             .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
             .median()
             .clip(aoi))
    return image.visualize(min=0, max=3000, bands=["B4", "B3", "B2"])

# 연도 목록
years = [2019, 2020, 2021, 2022, 2023]

# 지도로 시각화
Map = geemap.Map(center=[40.098217, 67.959741], zoom=16)

for year in years:
    vis_img = get_s2_rgb(year)
    Map.addLayer(vis_img, {}, f"{year} Summer RGB")

# 기준점 및 범위 시각화
Map.addLayer(point, {"color": "red"}, "Target Point")
# Map.addLayer(aoi, {"color": "blue"}, "AOI")

# 지도 출력
Map

Map(center=[40.098217, 67.959741], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

###인공위성 영상 저장 코드

In [ ]:
# 필요한 라이브러리 설치 (Colab에서만 주석 제거해서 실행)
# !pip install earthengine-api geemap --quiet

# 🌍 Earth Engine 인증 및 초기화
import ee
import geemap

ee.Authenticate()  # 최초 한 번만 필요
ee.Initialize(project='energylife-458603')  # ← 본인의 GCP 프로젝트 ID로 변경

# 관심 지점 및 영역 정의
point = ee.Geometry.Point([40.098217, 67.959741])
aoi = point.buffer(5000).bounds()

# Sentinel-2 영상 가져오기 함수
def get_s2_rgb(year):
    start = ee.Date(f"{year}-03-01")
    end = ee.Date(f"{year}-05-30")
    image = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
             .filterBounds(aoi)
             .filterDate(start, end)
             .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
             .median()
             .clip(aoi))
    # 시각화를 위한 RGB 변환 (B4=Red, B3=Green, B2=Blue)
    return image.visualize(min=0, max=3000, bands=["B4", "B3", "B2"])

# 시각화 및 저장할 연도 리스트
years = [2019, 2020, 2021, 2022, 2023]

# 지도에서 확인용 시각화
Map = geemap.Map(center=[40.098217, 67.959741], zoom=16)
Map.addLayer(point, {"color": "red"}, "Target Point")

for year in years:
    vis_img = get_s2_rgb(year)
    Map.addLayer(vis_img, {}, f"{year} Summer RGB")

# 이미지 로컬 저장 (수정된 버전)
for year in years:
    ee_img = get_s2_rgb(year)
    file_name = f'sentinel2_rgb_{year}.tif'
    print(f"Exporting {file_name} ...")

    geemap.ee_export_image(
        ee_img,
        filename=file_name,
        scale=10,
        region=aoi,
        file_per_band=False
    )

    print(f"✅ Saved: {file_name}")

# 지도 출력 (Notebook/Colab에서만 작동)
Map

In [ ]:
# 필수 라이브러리 설치
!pip install earthengine-api geemap --quiet

# Earth Engine 인증
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='energylife-458603')  # ← 여기에 정확한 ID 입력


### 고도 차이 시각화 코드

In [9]:
import geemap

# 관심 지점 설정
point = ee.Geometry.Point([67.959741, 40.098217])
aoi = point.buffer(5000).bounds()

# ASTER GDEM: 평균 DEM (2000~2011)
aster = ee.Image("NASA/ASTER_GED/AG100_003").select("elevation").clip(aoi)

# ALOS AW3D30: 최근 DEM (2006~2011) - ImageCollection → mosaic 사용
alos = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2") \
    .filterBounds(aoi) \
    .select("DSM") \
    .mosaic() \
    .clip(aoi)

# 고도 차이 계산
dem_diff = alos.subtract(aster).rename("DEM_Diff")

# 시각화 설정
vis_params = {
    'min': -10,
    'max': 10,
    'palette': ['blue', 'white', 'red']
}

# 지도 시각화
Map = geemap.Map(center=[40.098217, 67.959741], zoom=13)
Map.addLayer(dem_diff, vis_params, "DEM Difference (ALOS - ASTER)")
Map.addLayer(point, {"color": "yellow"}, "Target Point")
Map.addLayer(aoi, {"color": "green"}, "AOI (5km)")
Map

Map(center=[40.098217, 67.959741], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [10]:
# 🎯 매립지 위치 (Chiqindi poligoni)
point = ee.Geometry.Point([68.7372, 40.4061])
aoi = point.buffer(1500).bounds()

# 🛰️ SRTM DEM
srtm = ee.Image("USGS/SRTMGL1_003").clip(aoi)

# 📊 고도 min/max 값 추출
elevation_stats = srtm.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e8
).getInfo()

print("고도 통계값:", elevation_stats)

# 🎨 시각화용 범위 설정
srtm_vis = {
    "min": elevation_stats['elevation_min'],
    "max": elevation_stats['elevation_max'],
    "palette": ["blue", "green", "brown", "white"]
}

# 🗺️ 지도 생성
Map = geemap.Map(center=[40.4061, 68.7372], zoom=15)
Map.addLayer(srtm, srtm_vis, "SRTM DEM (Auto-Scaled)")
Map.addLayer(point, {"color": "red"}, "Target Point")
Map


고도 통계값: {'elevation_max': 274, 'elevation_min': 259}


Map(center=[40.4061, 68.7372], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

### 메탄 농도 관측 코드

In [ ]:
import ee, geemap
from datetime import datetime

# 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='energylife-458603')

# 관심 지점 및 영역 설정
point = ee.Geometry.Point([40.098217, 67.959741])
aoi = point.buffer(7000).bounds()  # Sentinel-5P 해상도(약 7km)에 맞춰 AOI 설정

# 날짜 설정
start_date = '2020-06-01'
end_date = '2020-06-30'

# Sentinel-5P 메탄 이미지 수집
collection = (
    ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4")
    .select("CH4_column_volume_mixing_ratio_dry_air")
    .filterBounds(aoi)
    .filterDate(start_date, end_date)
)

# 이미지 개수 확인
img_count = collection.size().getInfo()
print(f"해당 기간 내 이미지 개수: {img_count}")

# 실제 날짜 확인
dates = collection.aggregate_array('system:time_start').getInfo()
readable_dates = [datetime.utcfromtimestamp(t/1000).strftime('%Y-%m-%d') for t in dates]
print("관측된 날짜들:", readable_dates)

# 이미지가 있는 경우만 진행
if img_count > 0:
    # 평균 이미지 계산 (마스크 유지)
    mean_image = collection.mean().clip(aoi)

    # 평균 메탄 농도 계산
    mean_value = mean_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=7000,
        maxPixels=1e8
    ).get('CH4_column_volume_mixing_ratio_dry_air').getInfo()
    print("관심 영역의 평균 메탄 농도 (mol/m²):", mean_value)

    # 자동 시각화 범위 계산
    stats = mean_image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=aoi,
        scale=7000,
        maxPixels=1e8
    ).getInfo()
    ch4_min = stats.get('CH4_column_volume_mixing_ratio_dry_air_min')
    ch4_max = stats.get('CH4_column_volume_mixing_ratio_dry_air_max')

    # 시각화 범위 보정 (값이 너무 낮으면 기본값 사용)
    if ch4_min is None or ch4_min < 1000:
        ch4_min = 1600
    if ch4_max is None or ch4_max < ch4_min + 50:
        ch4_max = 2000

    print("자동 보정된 시각화 범위:", ch4_min, "~", ch4_max)

    # 시각화 설정
    ch4_vis = {
        'min': ch4_min,
        'max': ch4_max,
        'palette': ['blue', 'green', 'yellow', 'orange', 'red']
    }

    # 지도 시각화
    Map = geemap.Map(center=[40.4061, 68.7372], zoom=10)
    Map.addLayer(mean_image, ch4_vis, "Mean Methane (CH4)")
    Map.addLayer(point, {"color": "red"}, "Target Point")
    Map.addLayer(aoi, {"color": "blue"}, "AOI 범위")

    # 유효 픽셀 마스크 시각화 (디버깅용)
    mask_layer = mean_image.mask().reduce(ee.Reducer.min())
    Map.addLayer(mask_layer, {'min': 0, 'max': 1}, 'Valid CH4 Pixel Mask')


else:
    print("지정한 날짜 범위 내에 사용 가능한 메탄 이미지가 없습니다.")
Map

해당 기간 내 이미지 개수: 381
관측된 날짜들: ['2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-01', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-02', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-03', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-04', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '2020-06-05', '202

Map(center=[40.4061, 68.7372], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…